In [1]:
sc.install_pypi_package('pandas==1.0.3')
sc.install_pypi_package("boto3")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1654254954830_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/4a/6a/94b219b8ea0f2d580169e85ed1edc0163743f55aaeca8a44c2e8fc1e344e/pandas-1.0.3-cp37-cp37m-manylinux1_x86_64.whl

  Using cached https://files.pythonhosted.org/packages/26/b2/31748931b9682dcb4c09e8a15b9daeab1f434c90097425ca06e37d6efc7c/boto3-1.24.2-py3-none-any.whl

In [2]:
sc.addPyFile("s3://crpytoconven/project/supp/preprocessing.py")
import glob, os
import numpy as np
import boto3
from preprocessing import *
import pyspark.sql.functions as f
from pyspark.sql import Window
import pandas as pd
from pyspark.sql.functions import col, unix_timestamp, to_date, when, monotonically_increasing_id, coalesce, last, first
from pyspark.sql.types import DateType
import pyspark.sql.functions as f
import io
from io import StringIO 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# raw dataset
cryptocurrencies = spark.read.csv('s3://crpytoconven/project/data/coins_2015_2022.csv',header=True)
cryptocurrencies = cryptocurrencies[['symbol', 'open', 'high', 'low', 'close', 'time_open']]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
cryptocurrencies = cryptocurrencies.withColumnRenamed("symbol","Symbol") \
    .withColumnRenamed("open","Open") \
    .withColumnRenamed("high","High") \
    .withColumnRenamed("close","Close") \
    .withColumnRenamed("time_open","Date")

# cleaned reference
coins = spark.read.csv('s3://crpytoconven/project/data/crypto_2015_2022.csv',header=True) 
coins=coins.drop('_c0')

currs = coins.columns[1:]
cryp_dataset = {}
for curr in currs:
    sup = cryptocurrencies[cryptocurrencies.Symbol == curr]
    cryp_dataset[curr+'USD'] = sup

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# my data pipeline
myDataPip = DataPipline(start_date='2015-09-01', end_date='2022-05-01')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# fill dataset
for curr in cryp_dataset.keys():
    cryp_dataset[curr] = myDataPip.fill_dataset(dataset=cryp_dataset[curr], start_date='2015-09-01', end_date='2022-05-01' ,spark=spark, _type="crpyto")
    
# unify unit
new_cryp_dataset = {}
for curr in cryp_dataset.keys():
    dataset, name = myDataPip.unify_unit(cryp_dataset[curr], curr)
    new_cryp_dataset[name] = dataset
del cryp_dataset

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
crypto=new_cryp_dataset

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Activate the parallelized pyspark preocessing python script

In [8]:
# activate preprocessing procedure and save the processed sets of data to a dictionary!
data_dct = myDataPip.activate(crypto=new_cryp_dataset, spark=spark)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Processing complete!

### time: 5:44 --> 5:46  time elapsed: 2 min !

## Upload the processed pyspark dataframe as parquet files to s3 bucket

In [9]:
# convert the pyspark dataframe to parquet
# uploading the parquet files to s3 bucket! (It would take a while)
myDataPip.upload_s3(data_dct)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-9:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1779

An error was encountered:
An error occurred while calling o54639.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:226)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:181)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.sca

In [ ]:
#8:34 --> 7:31 (including upload parquet files to s3) --> 1 hour (15 parquets)